In [1]:
import pandas as pd 
import numpy as np
from sqlalchemy import create_engine

In [2]:
df = pd.read_csv("C:/Users/joth1/Documents/miniprojeto_pandas_sql/Miniprojeto-Pandas-e-SQL/credito.csv")


try:

    usuario = 'root'
    senha = '' # coloque sua senha aqui 
    host = 'localhost'
    porta = '3306'
    banco_de_dados = 'CREDITO'

    engine = create_engine(f'mysql+mysqlconnector://{usuario}:{senha}@{host}:{porta}/{banco_de_dados}')

    nm_tabela = 'credito_arq'
    df.to_sql(nm_tabela, engine, index=False, if_exists='replace')

    query = f'SELECT * FROM {nm_tabela}'
    
    output = pd.read_sql_query(query, engine)
    display(output)


except:
    display('Não foi possivel inserir no BD, verifique as configurações')



'Não foi possivel inserir no BD, verifique as configurações'

In [3]:
display(df.head(1))

,id,default,idade,sexo,dependentes,escolaridade,estado_civil,salario_anual,tipo_cartao,meses_de_relacionamento,qtd_produtos,iteracoes_12m,meses_inativo_12m,limite_credito,valor_transacoes_12m,qtd_transacoes_12m
0,768805383,0,45,M,3,ensino medio,casado,$60K - $80K,blue,39,5,3,1,"12.691,51","1.144,90",42


In [4]:
# arrumando variaveis
df['sexo'] = df['sexo'].replace({'M' : 'Masculino', 'F': 'Feminino'})


# criação das faixas 
df['faixa_qtd_produtos'] = np.where(df['qtd_produtos'] == 0, 'Sem prods'
                                    ,np.where(df['qtd_produtos'] > 5, ' > 5 produtos', '1-5 produtos'))

df['faixa_meses_relacionamento'] = np.where(df['meses_de_relacionamento'] < 12, 'Cliente Novo'
                                    ,np.where(df['meses_de_relacionamento']  < 18, 'Cliente operável', 'Cliente Senior'))


df['ds_cartao'] = np.where(df['tipo_cartao'] == 'blue', 'Cartao_blue', 'Outros')


# pega os cartôes diferentes de blue que são os clientes que queremos atuar 
display(df[df['ds_cartao'] == 'Outros'])




,id,default,idade,sexo,dependentes,escolaridade,estado_civil,salario_anual,tipo_cartao,meses_de_relacionamento,qtd_produtos,iteracoes_12m,meses_inativo_12m,limite_credito,valor_transacoes_12m,qtd_transacoes_12m,faixa_qtd_produtos,faixa_meses_relacionamento,ds_cartao
6,810347208,0,51,Masculino,4,na,casado,$120K +,gold,46,6,3,1,"34.516,72","1.330,87",31,> 5 produtos,Cliente Senior,Outros
7,818906208,0,32,Masculino,0,ensino medio,na,$60K - $80K,silver,27,2,2,2,"29.081,49","1.538,32",36,1-5 produtos,Cliente Senior,Outros
37,785432733,0,42,Feminino,4,ensino medio,casado,menos que $40K,gold,36,2,3,3,"15.433,35","966,80",22,1-5 produtos,Cliente Senior,Outros
42,773146383,0,50,Feminino,1,graduacao,solteiro,$40K - $60K,silver,43,3,3,2,"17.304,68","1.756,00",33,1-5 produtos,Cliente Senior,Outros
61,712030833,1,48,Masculino,2,mestrado,casado,$60K - $80K,silver,35,2,4,4,"34.516,60","691,10",15,1-5 produtos,Cliente Senior,Outros
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10100,719848008,0,39,Masculino,2,mestrado,na,$60K - $80K,silver,36,4,2,2,"29.808,17","16.098,13",128,1-5 produtos,Cliente Senior,Outros
10104,715474083,0,51,Masculino,3,mestrado,solteiro,$60K - $80K,silver,36,3,2,2,"29.663,66","14.638,23",93,1-5 produtos,Cliente Senior,Outros
10110,718225683,0,56,Masculino,1,mestrado,solteiro,$80K - $120K,silver,49,5,2,2,"34.516,38","15.274,57",108,1-5 produtos,Cliente Senior,Outros
10112,708564858,1,33,Masculino,2,graduacao,casado,$120K +,gold,20,2,4,1,"34.516,84","9.338,74",73,1-5 produtos,Cliente Senior,Outros


In [5]:
df2 = df.copy()

display(df2.dtypes)


tabela = df2.pivot_table(
    columns='faixa_meses_relacionamento', 
    index='escolaridade', 
    values='qtd_produtos'
    ,aggfunc='sum'
)

display(tabela)




id                             int64
default                        int64
idade                          int64
sexo                          object
dependentes                    int64
escolaridade                  object
estado_civil                  object
salario_anual                 object
tipo_cartao                   object
meses_de_relacionamento        int64
qtd_produtos                   int64
iteracoes_12m                  int64
meses_inativo_12m              int64
limite_credito                object
valor_transacoes_12m          object
qtd_transacoes_12m             int64
faixa_qtd_produtos            object
faixa_meses_relacionamento    object
ds_cartao                     object
dtype: object

faixa_meses_relacionamento,Cliente Senior,Cliente operável
escolaridade,,
doutorado,3626,73
ensino medio,7511,153
graduacao,3694,104
mestrado,11762,203
na,5647,122
sem educacao formal,5622,93
